In [6]:
import pandas as pd
import numpy as np
from datetime import date,timedelta,datetime
from functools import reduce

In [7]:
path_inicial_1 = "./datos/ingesta_pronostico_meteo.csv"
path_final = "./datos/ingesta_contaminacion.csv"

In [12]:
def dataset_aire(particula,lastHour):
    print(particula)
    mesAnterior = str( (date.today().replace(day=1)+timedelta(days=-1)).month ).zfill(2)
    yearAnterior = str( (date.today().replace(day=1)+timedelta(days=-1)).year )
        
    mesActual = str(date.today().month).zfill(2)
    yearActual = str(date.today().year)
    
    def dataset(part,anio,mes):
        df = pd.read_html("http://www.aire.cdmx.gob.mx/estadisticas-consultas/concentraciones/" \
        +"respuesta.php?qtipo=HORARIOS&parametro="+part+"&anio="+anio+"&qmes="+mes,\
        na_values=["nr"],keep_default_na=True, skiprows = 1  )
        df[0].columns = df[1].values.tolist()[0]
        df = df[0].drop(0,axis=0)
        return df
        
    
    dfAnterior = dataset(particula,yearAnterior,mesAnterior)  
    dfActual = dataset(particula,yearActual,mesActual)
    df = pd.concat([dfAnterior,dfActual])
    df = df.tail(int(lastHour))
    
    namecolum = df.columns.tolist()                              
    fecha = namecolum.pop(0)
    hora = namecolum.pop(0)
    df = pd.melt(df, id_vars=[fecha, hora], value_vars=namecolum,var_name="id_station", value_name=particula.upper())
    df.loc[::,fecha] = df[fecha].apply(lambda x: datetime.strptime(x, '%d-%m-%Y'))
    df.columns = [df.columns[0].lower(), df.columns[1].lower(),df.columns[2],df.columns[3] ]
    return df

In [13]:
particulas = ['so2','co','nox','no','no2','o3','pm10','wsp','wdr','tmp','rh','pm2']

In [14]:
lastHour = 30000
listaContaminantes = [dataset_aire(particula,lastHour ) for particula in particulas]
listNameJoin = listaContaminantes[0].columns.tolist()
listNameJoin.pop(3)

so2
co
nox
no
no2
o3
pm10
wsp
wdr
tmp
rh
pm2


'SO2'

In [15]:
df_final = reduce(lambda left,right: pd.merge(left,right,on=listNameJoin,how="outer"), listaContaminantes)

1. Poner atributo dia, mes
2. arreglar fechas
3. fecha en formato YYYY-MM-dd HH:mm


Convertir la hor ay fecha de cantidad de horas a uso horario

In [16]:
def time_converter(hora,fecha):
    if hora[:].endswith("24"):
        fecha = str(fecha+" "+str(int(hora)-1)+":00")
        the_time = datetime.strptime(fecha, '%Y-%m-%d %H:%M')
        new_time = the_time + timedelta(hours=1)
        return new_time.strftime('%Y-%m-%d'),str(int(new_time.strftime('%H')))
    else:
        return  fecha, hora

In [17]:
df_final["fecha"] = df_final["fecha"].astype(str)
df_final["hora"] = df_final["hora"].astype(str)
df_final["fecha"] = df_final.apply(lambda row: time_converter(row["hora"],row["fecha"]), axis=1).str.get(0)
df_final["hora"] = df_final.apply(lambda row: time_converter(row["hora"],row["fecha"]), axis=1).str.get(1)

convertir fecha a timestand, calcular en otra columna mes y dia

In [21]:
df_final["fecha"] = (df_final["fecha"] +" "+df_final["hora"]).apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H'))
df_final["mes"] = df_final["fecha"].apply(lambda fecha: fecha.month )
df_final["dia"] = df_final["fecha"].apply(lambda fecha: fecha.day )

reordenar columnas

In [23]:
df_final = df_final.loc[::, ['fecha', 'mes', 'dia', 'hora', 'id_station', 'SO2', 'CO', 'NOX', 'NO', 'NO2', 'O3',
       'PM10', 'WSP', 'WDR', 'TMP', 'RH', 'PM2'] ]

## Merge con Datos de pronóstico

In [36]:
path_final = "./datos/ingesta_contaminacion.csv"

df1 =  pd.read_csv(path_inicial_1)

df = pd.concat( [ df1, df_final ],sort=True )

df = df.sort_values(["id_station","fecha"],ascending=False).reset_index(drop=True)

df = df.rename({"PM2":"PM2.5"},axis=1)

df = df.drop(["WDR"],axis=1)


In [40]:
df.to_csv(path_final,index=False)